Copyright 2023 Province of British Columbia

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at 

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under he License.
the License.

In [ ]:
import pandas as pd
from autocorrect import Speller

# add our stuff to the path
import sys
import os
import re
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# import our stuff
from importlib import reload
from src import matching, connect

import warnings
warnings.simplefilter(action='ignore')

In [ ]:
# PARAMETERS

# info to gain access to database, IDIR restricted 
CRED_PATH = '../credentials.txt'

# which tables to access
RESPONSE_TABLE = 'dbo.AQ29LANG'
CODE_TABLE = 'dbo.AQ29LANG_Codes'
RESULTS_TABLE = 'dbo.AQ29LANG_RESULTS'

# Which Cycle to work on 
CYCLE = 3

In [ ]:
reload(matching)

## Read in Data

In [ ]:
# Read in all data required to build model
connection = connect.create_connection(CRED_PATH)

# actual responses
df_open = connect.fetch_table(RESPONSE_TABLE, connection)

# codes to match
df_codes = connect.fetch_table(CODE_TABLE, connection)

In [ ]:
df_open.head()

In [ ]:
df_open.shape

In [ ]:
df_codes.head()

In [ ]:
# for q29, some of the descriptions are hidden in the desc column - pull these out
df_codes_dict = {
    'q_code': [],
    'qc_desc': []
}

for idx, row in df_codes.iterrows():
    q_code = row.q_code
    qc_desc = row.qc_desc

    # roll indigenous bc languages into the same code
    note = row.additional_notes
    if note == 'Roll into 6':
        q_code = '6'

    # add portugalês in manually
    if qc_desc == 'Portuguese':
        df_codes_dict['q_code'].append(q_code)
        df_codes_dict['qc_desc'].append('portugalês')

    # split the qc desc to remove the word 'languages' 
    if qc_desc == 'Indigenous languages in B.C.':
        df_codes_dict['q_code'].append(q_code)
        df_codes_dict['qc_desc'].append(qc_desc)

    else:
        if qc_desc is not None:
            code_list = matching.split_languages(qc_desc)
            for code in code_list:
                df_codes_dict['q_code'].append(q_code)
                df_codes_dict['qc_desc'].append(code)

    # get the responses from the optional row as well 
    qc_optional = row.qc_desc_notes
    if not pd.isnull(qc_optional):
        code_list = matching.split_languages(qc_optional)
        for code in code_list:
            df_codes_dict['q_code'].append(q_code)
            df_codes_dict['qc_desc'].append(code)
            

# add some extras
df_codes_dict['q_code'].append('1222101')
df_codes_dict['qc_desc'].append('ASL')

# remove duplicates
df_codes_updated = pd.DataFrame(df_codes_dict)
df_codes_updated = df_codes_updated.groupby('qc_desc').first().reset_index()

In [ ]:
df_codes_updated

In [ ]:
code_list = df_codes_updated.qc_desc.values
code_list

In [ ]:
# grab only the cycle of current interest
df = df_open[df_open.cycle == CYCLE].reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['response'] = df['aq29lang']

In [ ]:
spell = Speller()

# get a list of all words that are directly in the code words 
all_words = []
for word in code_list:
    words = re.split(r"\sand\s|[,;()/\r\n\s'-]+", word)
    for x in words:
        if len(x) > 0:
            all_words.append(x)
            
# add additional words that are not inaccurate 
words = [
]

for word in all_words + words:

    for x in [word, word.upper(), word.lower()]:
        if x in spell.nlp_data:
            continue
            
        spell.nlp_data[word] = 1_000_000
        spell.nlp_data[word.upper()] = 1_000_000
        spell.nlp_data[word.lower()] = 1_000_000

## Clean Data

Transformation Steps:

1. Clean html codes, remove trailing spaces
2. Do a basic spell check, but include unusual words from our word list to make sure they don't incorrectly change
3. Translate a subset of the responses - this takes a long time so only those that start with & (and are thus most likely a different language) are translated.
4. Determine if the translated response is an exact or partial match to any of the codes in the code list
5. For partial responses, remove as a possibility if it is just a subset of an exact response
6. Tabulate all information for each response

In [ ]:
reload(matching)

In [ ]:
clean_dict = {
    'id': [],
    'cycle': [],
    'response': [], 
    'cleaned': [],
    'translated': [],
    'translation_code': [],
    'exact_match': [],
    'partial_match': [],
    'exact_match_codes': [],
    'partial_match_codes': [],
    'likely_match_codes': [],
    'q29lang_c01': [],
    'q29lang_c02': [],
    'q29lang_c03': [],
    'q29lang_c04': []
}

n_rows = df.shape[0]

for idx, row in df.iterrows():

    x = row.response
    id = row.id
    cycle = row.cycle
    (response, 
     cleaned, translated, response_code, 
     has_exact, has_partial, 
     exact_match_codes, partial_match_codes, likely_match_codes) = matching.do_the_things(x, spell, code_list, translate_all = False)

    clean_dict['id'].append(id)
    clean_dict['cycle'].append(cycle)
    clean_dict['response'].append(response)    
    clean_dict['cleaned'].append(cleaned)
    clean_dict['translated'].append(translated)
    clean_dict['translation_code'].append(response_code)
    clean_dict['exact_match'].append(has_exact)
    clean_dict['partial_match'].append(has_partial)
    clean_dict['exact_match_codes'].append(exact_match_codes)
    clean_dict['partial_match_codes'].append(partial_match_codes)
    clean_dict['likely_match_codes'].append(likely_match_codes)

    # code version of new codes 
    new_codes = []
    split_codes = likely_match_codes.split(', ')
    for code in split_codes:
        if code != '':
            new_codes.append(df_codes_updated[df_codes_updated.qc_desc.str.lower() == code.lower()].q_code.values[0])

    # remove 97 from list of codes included
    existing_codes = []
    for ii in range(1,5):
        col_name = f'q29lang_c0{ii}'
        val = row[col_name]
        if not pd.isnull(val):
            if val!='97':
                existing_codes.append(val)

    # add new codes into code columns
    all_codes = existing_codes + new_codes 
    for jj in range(1, 5):
        col_name = f'q29lang_c0{jj}'
        if jj <= len(all_codes):
            clean_dict[col_name].append(all_codes[jj-1])
        else:
            clean_dict[col_name].append(None)
    
    pct_done = int(round(100*(idx+1)/n_rows))
    print_line = f'{idx+1:07,}/{n_rows:07,}   |' + '-'*(pct_done) + '>' + ' '*(100-pct_done) + '|'
    print(print_line, end = '\r')

In [ ]:
clean_df = pd.DataFrame(clean_dict)

In [ ]:
# make sure we are in the right order
clean_df = clean_df[
    [
        'id', 
        'cycle',
        'response',
        'cleaned',
        'translated',
        'translation_code',
        'exact_match',
        'partial_match',
        'exact_match_codes',
        'partial_match_codes',
        'likely_match_codes',
        'q29lang_c01',
        'q29lang_c02',
        'q29lang_c03',
        'q29lang_c04',
    ]
]

In [ ]:
clean_df

In [ ]:
exact = clean_df.exact_match.sum()
n_rows = clean_df.shape[0]
partial_no_exact = clean_df[~clean_df.exact_match].partial_match.sum()
n_rows_no_exact = clean_df[~clean_df.exact_match].shape[0]
print(f'Exact Matches: {exact:,}/{n_rows:,} ({exact/n_rows:.0%})')
print(f'Partial Matches: {partial_no_exact:,}/{n_rows_no_exact:,} ({partial_no_exact/n_rows_no_exact:.0%})')
print(f'Leftover: {n_rows - exact - partial_no_exact:,}/{n_rows:,} ({(n_rows - exact - partial_no_exact)/n_rows:.0%})')

In [ ]:
# send back to warehouse 
engine = connect.create_connection(CRED_PATH, sqlalchemy=True)
connect.save_table(clean_df, RESULTS_TABLE, engine, how='append')

## Get Frequencies of Common 'Other' Response Words

In [ ]:
leftover = clean_df[~clean_df.exact_match & ~clean_df.partial_match].translated.values

In [ ]:
clean_df[~clean_df.exact_match & ~clean_df.partial_match]

In [ ]:
frequencies = matching.tokenize_and_count_word_frequencies(leftover)

In [ ]:
frequencies

In [ ]:
df_freq = pd.DataFrame(data = [frequencies.keys(), frequencies.values()], index = ['word', 'frequency']).T

In [ ]:
df_freq = df_freq.sort_values(by='frequency', ascending=False)

In [ ]:
# send frequencies back for possible inclusion into codes
engine = connect.create_connection(CRED_PATH, sqlalchemy=True)
connect.save_table(df_freq, 'dbo.AQ29LANG_FREQUENCIES', engine, how='replace')